In [1]:
!pip install -U autogluon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [2]:
!wget https://raw.githubusercontent.com/autogluon/autogluon/master/examples/automm/kaggle_california_house_price/example_kaggle_house.py


--2025-10-14 02:50:41--  https://raw.githubusercontent.com/autogluon/autogluon/master/examples/automm/kaggle_california_house_price/example_kaggle_house.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6053 (5.9K) [text/plain]
Saving to: ‘example_kaggle_house.py’

example_kaggle_hous 100%[===================>]   5.91K  --.-KB/s    in 0s      

2025-10-14 02:50:42 (67.3 MB/s) - ‘example_kaggle_house.py’ saved [6053/6053]



In [3]:
!wget https://raw.githubusercontent.com/autogluon/autogluon/master/examples/automm/kaggle_california_house_price/README.md
!cat README.md


--2025-10-14 02:50:52--  https://raw.githubusercontent.com/autogluon/autogluon/master/examples/automm/kaggle_california_house_price/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1729 (1.7K) [text/plain]
Saving to: ‘README.md’

README.md           100%[===================>]   1.69K  --.-KB/s    in 0s      

2025-10-14 02:50:52 (21.5 MB/s) - ‘README.md’ saved [1729/1729]

# Use AutoMM to Predict California House Prices

Example shows how to combine the tabular deep learning (DL) model in AutoMM and other tree models via the auto-ensembling logic in AutoGluon-Tabular 
to train a good model on the [Kaggle: California House Price Competition](https://www.kaggle.com/c/california-house-prices).

The task is to predict house sale prices based on the hous

In [4]:
!python example_kaggle_house.py



2025-10-14 02:51:27.078258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760410287.362748    2142 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760410287.442328    2142 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760410288.019202    2142 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760410288.019253    2142 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760410288.019258    2142 computation_placer.cc:177] computation placer alr

In [5]:
!ls


example_kaggle_house.py  README.md  sample_data


In [7]:
!cat example_kaggle_house.py


import pandas as pd
import numpy as np
import argparse
import os
import random
from autogluon.tabular import TabularPredictor
from autogluon.multimodal import MultiModalPredictor
import torch as th


def get_parser():
    parser = argparse.ArgumentParser(
        description='The Basic Example of AutoGluon for House Price Prediction.')
    parser.add_argument('--mode',
                        choices=['stack5',
                                 'weighted',
                                 'single',
                                 'single_bag5'],
                        default='weighted',
                        help='"stack5" means 5-fold stacking. "weighted" means weighted ensemble.'
                             ' "single" means use a single model.'
                             ' "single_bag5" means 5-fold bagging via the AutoMM model.')
    parser.add_argument('--automm-mode', choices=['ft-transformer', 'mlp'],
                        default='ft-transformer', help='Fusion model in 